# 🌿 Bitki Model → TF.js (ULTIMATE FIX)

**Strateji:** Modeli H5 formatında kaydet → TF.js'e dönüştür

In [ ]:
!pip install tensorflowjs h5py -q

import tensorflow as tf
import keras
import numpy as np
import os

print(f"✅ TensorFlow: {tf.__version__}")
print(f"✅ Keras: {keras.__version__}")

In [ ]:
from google.colab import files

print("📁 plants_best_model.keras dosyasını yükleyin:")
uploaded = files.upload()
model_file = list(uploaded.keys())[0]

print(f"\n📂 Model yükleniyor...")
model = keras.saving.load_model(model_file)

print(f"✅ Input: {model.input_shape}")
print(f"✅ Output: {model.output_shape} ({model.output_shape[-1]} sınıf)")

In [ ]:
# Functional API ile yeni model oluştur (augmentation olmadan)
print("🔧 Inference modeli oluşturuluyor...\n")

# Yeni input
new_input = keras.Input(shape=(224, 224, 3), name='image_input')

# Model'i inference modunda çağır
output = model(new_input, training=False)

# Yeni model
inference_model = keras.Model(inputs=new_input, outputs=output, name='inference_model')

print(f"✅ Inference model oluşturuldu")
print(f"   Input: {inference_model.input_shape}")
print(f"   Output: {inference_model.output_shape}")

# Test
test_img = np.random.rand(1, 224, 224, 3).astype(np.float32)
test_out = inference_model.predict(test_img, verbose=0)
print(f"\n🧪 Test başarılı! Output shape: {test_out.shape}")

In [ ]:
# H5 formatında kaydet
h5_file = "model_for_tfjs.h5"

print(f"💾 H5 formatında kaydediliyor: {h5_file}")
inference_model.save(h5_file, save_format='h5')

print(f"✅ Kaydedildi ({os.path.getsize(h5_file)/1024/1024:.1f} MB)")

In [ ]:
# TF.js'e dönüştür
output_dir = "tfjs_model"

print("🔄 TensorFlow.js'e dönüştürülüyor...\n")

!tensorflowjs_converter \
    --input_format=keras \
    --output_format=tfjs_graph_model \
    {h5_file} \
    {output_dir}

print("\n📁 Oluşturulan dosyalar:")
total = 0
for f in sorted(os.listdir(output_dir)):
    size = os.path.getsize(os.path.join(output_dir, f))
    total += size
    print(f"   {f} ({size/1024/1024:.1f} MB)")
print(f"\n📦 Toplam: {total/1024/1024:.1f} MB")

In [ ]:
# ZIP ve indir
import shutil

shutil.make_archive('tfjs_model', 'zip', output_dir)

print("📥 İndiriliyor...")
files.download('tfjs_model.zip')

print("\n" + "="*60)
print("🎉 BAŞARILI!")
print("="*60)
print("\n1. ZIP'i açın")
print("2. Dosyaları BitkiTanima/assets/tfjs_model/ klasörüne kopyalayın")
print("3. modelService.ts'de DEMO_MODE = false yapın")
print("4. npx expo start --clear")
print("5. Test edin! 🚀")